In [1]:
# code is from below, pytorch torchvision github
# https://github.com/pytorch/vision/tree/main/torchvision/models/detection

In [2]:
import numpy as np
import os
import random
import csv
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import cv2
import albumentations as A
import albumentations.pytorch
from sklearn.model_selection import train_test_split

import sys
sys.path.append('../')
sys.path.append('../../data/')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
import torch.optim as optim

from _utils import warmup_lr_scheduler, reduce_dict
from dataset import LbpDataset, train_transforms, val_transforms, test_transforms, collate_fn, get_data
# from loss import LBPloss
# from engine import train_one_epoch, evaluate
from visualize import visualize
from model import fasterrcnn_resnet101_fpn, fasterrcnn_resnet152_fpn, fasterrcnn_resnet18_fpn
from anchor_utils import *
from rpn import *
from roi_heads import *


In [3]:
from faster_rcnn import FastRCNNPredictor

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [5]:
num_classes = 2
model = fasterrcnn_resnet18_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.train()
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(2048,), max_size=2048, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 

In [6]:
# model.eval()

In [7]:
df = pd.read_csv('../../data/df.csv')
df.shape

(6736, 11)

In [8]:
df.head()

,file_name,task,bbox,xmin,ymin,w,h,label,occluded,des,cell_type
0,patch_images/2021.01.12/LBC305-20210108(1)/LBC...,[ASCUS] LBC305,"[56, 35, 1980, 1985]",56,35,1980,1985,판독불가,0,NaN,NaN
1,patch_images/2021.01.12/LBC305-20210108(1)/LBC...,[ASCUS] LBC305,"[56, 30, 1912, 1937]",56,30,1912,1937,판독불가,0,NaN,NaN
2,patch_images/2021.01.12/LBC305-20210108(1)/LBC...,[ASCUS] LBC305,"[21, 12, 2010, 2027]",21,12,2010,2027,판독불가,0,NaN,NaN
3,patch_images/2021.01.06/LBC37-20210102(1)/LBC3...,[ASCUS] LBC37,"[1349, 420, 100, 113]",1349,420,100,113,ASC-US,0,NaN,Atypical squamous cells of undetermined signif...
4,patch_images/2021.01.06/LBC37-20210102(1)/LBC3...,[ASCUS] LBC37,"[1575, 720, 163, 213]",1575,720,163,213,ASC-US,0,NaN,Atypical squamous cells of undetermined signif...


In [9]:
# df.label.value_counts()

In [10]:
df['label_id'] = df.label.apply(lambda x : 1 if 'ASC-US' in x or 'ASC-US with HPV infection' in x 
                                or 'AS' in x else 0.)
df = df[df['label_id'] == 1]

In [11]:
df.label.value_counts()

AS                           1947
ASC-US                       1931
ASC-US with HPV infection     724
ASC-H                          75
ASCUS-SIL                       2
Name: label, dtype: int64

In [12]:
# df['xmin']

In [13]:
# df['xdiff'] = df.apply(lambda x : x['xmax'] - x['xmin'], axis=1)
df['xmax'] = df.apply(lambda x : x['xmin'] + x['w'], axis=1)
df['ymax'] = df.apply(lambda x : x['ymin'] + x['h'], axis=1)
df = df[['file_name', 'task', 'bbox', 'xmin', 'ymin', 'xmax', 'ymax', 'w', 'h', 'label',
       'occluded', 'des', 'cell_type', 'label_id']]
# df.head()

In [14]:
df_group = df.groupby('file_name')
df_list = df.file_name.unique()
train_list, test_list = train_test_split(df_list, test_size=0.2, random_state=42)
print('total {} train {} test {}'.format(len(df_list), len(train_list), len(test_list)))

train_list = [get_data(img_id, df_group) for img_id in train_list]
test_list = [get_data(img_id, df_group) for img_id in test_list]

print(len(train_list))
print(len(test_list))
train_list[0]

total 4019 train 3215 test 804
3215
804


{'image_id': 'patch_images/2021.01.14/LBC426-20210111(1)/LBC426-20210111(1)_1005.png',
 'boxes': array([[1728,  231, 1868,  365],
        [1948,   85, 2047,  283],
        [1354, 1778, 1513, 1951]]),
 'labels': array([1., 1., 1.]),
 'size': array([[140, 134],
        [ 99, 198],
        [159, 173]])}

In [15]:
BATCH_SIZE = 2
train_dataset = LbpDataset(
    train_list,
    transform=train_transforms,
)
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=8,
#     pin_memory=config.PIN_MEMORY,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)

test_dataset = LbpDataset(
    test_list,
    transform=val_transforms,  
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=8,
#     pin_memory=config.PIN_MEMORY,
    shuffle=True,
    drop_last=False,
    collate_fn=collate_fn
)

In [16]:
images, targets, path = next(iter(train_loader))
# print(path)
print(images[0].shape)
targets

torch.Size([3, 2048, 2048])


({'boxes': tensor([[1157.,  386., 1442.,  674.]]), 'labels': tensor([1])},
 {'boxes': tensor([[ 604.0713,    0.0000,  821.0779,  229.3168],
          [ 873.2430,  437.4984, 1090.2496,  675.2693]]),
  'labels': tensor([1, 1])})

In [17]:
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in list(targets)]

In [18]:
targets

[{'boxes': tensor([[1157.,  386., 1442.,  674.]], device='cuda:0'),
  'labels': tensor([1], device='cuda:0')},
 {'boxes': tensor([[ 604.0713,    0.0000,  821.0779,  229.3168],
          [ 873.2430,  437.4984, 1090.2496,  675.2693]], device='cuda:0'),
  'labels': tensor([1, 1], device='cuda:0')}]

In [19]:
images, targets = model.transform(images, targets)

In [20]:
images


In [21]:
features = model.backbone(images.tensors)

/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [22]:
proposals, proposal_losses = model.rpn(images, features, targets)

In [23]:
len(proposals[0])

2000

In [24]:
proposals, matched_idxs, labels, regression_targets = model.roi_heads.select_training_samples(proposals, targets)

In [31]:
# regression_targets

In [33]:
box_features = model.roi_heads.box_roi_pool(features, proposals, images.image_sizes)

In [35]:
box_features.shape

torch.Size([1024, 256, 7, 7])

In [24]:
detections, detector_losses = model.roi_heads(features, proposals, images.image_sizes, targets)

In [25]:
detections

[]

In [22]:
features['3'].shape

torch.Size([2, 256, 64, 64])

In [23]:
# list(features.values())

In [24]:
model.rpn.head

RPNHead(
  (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (cls_logits): Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
  (bbox_pred): Conv2d(256, 12, kernel_size=(1, 1), stride=(1, 1))
)

In [22]:
features = list(features.values())
objectness, pred_bbox_deltas = model.rpn.head(features)

In [23]:
(objectness[4].shape)

torch.Size([2, 3, 32, 32])

In [24]:
pred_bbox_deltas[4].shape

torch.Size([2, 12, 32, 32])

In [25]:
anchors = model.rpn.anchor_generator(images, features)

In [26]:
len(anchors[1])

1047552

In [27]:
num_images = len(anchors)
num_anchors_per_level_shape_tensors = [o[0].shape for o in objectness]
num_anchors_per_level = [s[0] * s[1] * s[2] for s in num_anchors_per_level_shape_tensors]
objectness, pred_bbox_deltas = \
    concat_box_prediction_layers(objectness, pred_bbox_deltas)

In [28]:
([s[0] * s[1] * s[2] for s in num_anchors_per_level_shape_tensors])

[786432, 196608, 49152, 12288, 3072]

In [29]:
proposals = model.rpn.box_coder.decode(pred_bbox_deltas.detach(), anchors)
proposals = proposals.view(num_images, -1, 4)

In [30]:
(proposals.shape)

torch.Size([2, 1047552, 4])

In [31]:
boxes, scores = model.rpn.filter_proposals(proposals, objectness, images.image_sizes, num_anchors_per_level)

In [32]:
len(boxes[0])

2000

In [44]:
labels, matched_gt_boxes = model.rpn.assign_targets_to_anchors(anchors, targets)

In [49]:
regression_targets = model.rpn.box_coder.encode(matched_gt_boxes, anchors)

In [51]:
regression_targets[0].shape

torch.Size([1047552, 4])

In [61]:
sampled_pos_inds, sampled_neg_inds = model.rpn.fg_bg_sampler(labels)
sampled_pos_inds = torch.where(torch.cat(sampled_pos_inds, dim=0))[0]
sampled_neg_inds = torch.where(torch.cat(sampled_neg_inds, dim=0))[0]
sampled_inds = torch.cat([sampled_pos_inds, sampled_neg_inds], dim=0)

In [62]:
len(sampled_inds)

512

In [ ]:
loss_objectness, loss_rpn_box_reg = self.compute_loss(objectness, pred_bbox_deltas, labels, regression_targets)

In [63]:
pred_bbox_deltas[sampled_pos_inds]

tensor([[-0.1205,  0.0254, -0.0594,  0.0458],
        [-0.1216,  0.0255, -0.0607,  0.0448],
        [-0.1218,  0.0254, -0.0602,  0.0453],
        [-0.1208,  0.0260, -0.0599,  0.0454],
        [-0.1016, -0.0714, -0.0263,  0.0643],
        [-0.1007, -0.0704, -0.0283,  0.0640],
        [-0.1002, -0.0727, -0.0257,  0.0639],
        [-0.0990, -0.0717, -0.0274,  0.0627]], device='cuda:0',
       grad_fn=<IndexBackward>)

In [27]:
# features

In [30]:
proposals, proposal_losses = model.rpn(images, features, targets)

In [35]:
len(proposals[1])

2000

In [32]:
proposal_losses

{'loss_objectness': tensor(0.6975, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(0.0091, device='cuda:0', grad_fn=<DivBackward0>)}

In [36]:
detections, detector_losses = model.roi_heads(features, proposals, images.image_sizes, targets)

In [37]:
detections

[]

In [38]:
detector_losses

{'loss_classifier': tensor(0.7143, device='cuda:0', grad_fn=<NllLossBackward>),
 'loss_box_reg': tensor(0.0004, device='cuda:0', grad_fn=<DivBackward0>)}

In [16]:
import torchvision
from faster_rcnn import FastRCNNPredictor
# from mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    
#     model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    model = fasterrcnn_resnet152_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

#     # now get the number of input features for the mask classifier
#     in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
#     hidden_layer = 256
#     # and replace the mask predictor with a new one
#     model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
#                                                        hidden_layer,
#                                                        num_classes)

    return model

In [17]:
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# # For Training
# images,targets, path = next(iter(train_loader))
# # print(image[0].shape)
# images = list(image for image in images)
# targets = [{k: v for k, v in t.items()} for t in list(targets)]
# output = model(images,targets)   # Returns losses and detections
# # For inference
# model.eval()
# x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# predictions = model(x)     

In [18]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')

In [19]:
num_classes = 2
num_epochs = 80
saved_model = '../trained_models/resnet_2048/'

model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.1,
                            momentum=0.9, weight_decay=0.0005)
#optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4 )

# # and a learning rate scheduler
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
#                                                step_size=3,
#                                                gamma=0.1)

In [20]:
#model.eval()


In [21]:
# evaluate(model, test_loader, device=device)

In [22]:
for epoch in range(num_epochs):
    model.train()

#     adjust_learning_rate(optimizer, epoch)
    EPOCH_lr = optimizer.param_groups[0]["lr"]
    
    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(train_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)
        
    batch_losses = []
    loop = tqdm(train_loader, leave=True)
    for images, targets, path in loop :
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in list(targets)]
#         print(targets[0]['labels'])
#         images, targets = images.to(device), targets.to(device)

        loss_dict = model(images,targets)
        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        optimizer.zero_grad()
        losses.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        batch_losses.append(losses.item())
        loop.set_postfix(epoch_loss=sum(batch_losses) / len(batch_losses), 
                         loss=losses.item(), lr=EPOCH_lr) 
        
    if epoch % 10 == 9 : 
        state = {
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }        
        torch.save(state, saved_model + 'epoch_' + str(epoch) +'_model.pt')        
        
#     evaluate(model, test_loader, device=device)

    if lr_scheduler is not None:
        lr_scheduler.step()    

  0%|          | 0/1757 [00:00<?, ?it/s]/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 1757/1757 [27:34<00:00,  1.06it/s, epoch_loss=0.198, loss=0.195, lr=0.0002] 


In [23]:
# state = {
#     'epoch': epoch,
#     'state_dict': model.state_dict(),
#     'optimizer': optimizer.state_dict(),
# }        
# torch.save(state, saved_model + 'epoch_' + str(epoch) +'_model.pt')   

In [24]:
# saved_model = '../trained_models/'
# device = torch.device('cpu')
state = torch.load(saved_model  + 'epoch_' + str(59) + '_model.pt')
model.load_state_dict(state['state_dict'])
# model.load_state_dict(state['optimizer'])
model.eval()
print('dd')

dd


In [25]:
images, targets, paths = next(iter(test_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in list(targets)]
model.eval()
with torch.no_grad():
    predictions = model(images)

In [1]:
NUM = 0
threshold = 0.3
image = images[NUM].permute(1,2,0).cpu().numpy() * 255.
pred_scores = predictions[NUM]['scores'].detach().cpu().numpy()
pred_bbox = predictions[NUM]['boxes'][:].detach().cpu().numpy()
# print(pred_bbox)
boxes = []
print('highest score', pred_scores[0:2])
for i, s in enumerate(pred_scores) :
    if s > threshold :
        boxes.append(pred_bbox[i])
    else :
        break

visualize(image , bboxes=boxes)

NameError: name 'images' is not defined

In [2]:
img_path = paths[NUM]
abs_path = '/home/Dataset/scl/'

train_image = cv2.imread(abs_path + img_path)
train_image = cv2.cvtColor(train_image, cv2.COLOR_BGR2RGB)
boxes = get_data(img_path, df_group)['boxes']
# boxes = targets[NUM]['boxes'].cpu().numpy()
print(boxes)
# boxes[:,2] = boxes[:,0] + boxes[:,2]
# boxes[:,3] = boxes[:,1] + boxes[:,3]

visualize(train_image, boxes[:,:4])

NameError: name 'paths' is not defined